In [1]:
import pandas as pd
import numpy as np

clients = pd.read_csv('../data/processed/clients.csv')
orders = pd.read_csv('../data/processed/orders.csv')
orders_raw = pd.read_csv('../data/processed/orders_raw.csv')

In [7]:
clients

,client_id,city,channel,promotor_id,frequency,efficiency,total_orders,total_volume,total_income,total_cost,...,efficiency_min,efficiency_max,efficiency_std,Class,avg_orders_per_month,avg_visits_per_month,visit_order_gap,inefficiency_cost,profit_per_visit,opportunity_cost
0,100006690,Madrid,AR,275609911,2.0,1.00,22,1658.706,1494.53,880,...,0.50,1.5,0.417424,LowTicket_Inefficient,1.833333,0.166667,-1.666667,-25.00,307.265,-512.108333
1,100008050,Barcelona,AR,368568690,1.0,0.00,14,3982.000,1905.59,140,...,0.00,0.0,0.000000,HighTicket_Inefficient,1.166667,0.000000,-1.166667,-17.50,0.000,-0.000000
2,100042162,Barcelona,HR,455263770,1.0,0.25,14,1812.850,2243.30,980,...,0.25,0.5,0.116775,HighTicket_Inefficient,1.166667,0.333333,-0.833333,-12.50,315.825,-263.187500
3,100046227,Barcelona,AR,454554895,4.0,2.00,16,4590.180,2273.12,640,...,2.00,2.0,0.000000,HighTicket_Efficient,1.333333,0.166667,-1.166667,-17.50,816.560,-952.653333
4,100125158,Cadiz,HR,483340469,3.0,3.00,25,1266.500,2204.24,625,...,1.00,4.0,0.971825,HighTicket_Efficient,2.083333,0.083333,-2.000000,-30.00,1579.240,-3158.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41945,999934164,Barcelona,HR,480416490,2.0,2.00,23,691.000,785.30,575,...,1.00,4.0,1.236033,LowTicket_Efficient,1.916667,0.083333,-1.833333,-27.50,210.300,-385.550000
41946,999940211,Barcelona,AR,908993212,1.0,0.00,3,557.820,260.55,30,...,0.00,0.0,0.000000,LowTicket_Inefficient,0.250000,0.000000,-0.250000,-3.75,0.000,-0.000000
41947,999940578,Madrid,AR,275609911,1.0,0.50,13,1101.524,1044.61,520,...,0.50,1.0,0.202260,LowTicket_Inefficient,1.083333,0.166667,-0.916667,-13.75,262.305,-240.446250
41948,999941988,Madrid,AR,677360818,3.0,1.50,36,5415.150,3828.31,1410,...,1.00,2.0,0.369274,HighTicket_Efficient,3.000000,0.166667,-2.833333,-42.50,1209.155,-3425.939167


In [20]:
clients[clients['efficiency'] > 10][['client_id', 'efficiency', 'frequency','median_promotor_visits']]

,client_id,efficiency,frequency,median_promotor_visits
6390,238243563,18.0,18.0,1.0
